In [1]:
import torch
import sys
sys.path.insert(0, "../..")
from src.data import data_tools, make_dataset
from pathlib import Path
from loguru import logger

2022-05-19 19:50:43.474268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 19:50:43.474306: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data_dir = "../../data/raw"
image_folder = make_dataset.get_flowers(data_dir)
dataloader = data_tools.Dataloader(path=image_folder, split=0.8)

2022-05-19 19:50:45.895 | INFO     | src.data.make_dataset:get_flowers:74 - Data is downloaded to ../../data/raw/datasets/flower_photos.


In [3]:
len(dataloader)

3670

In [4]:
dataloader.class_dict

{'roses': 0, 'tulips': 1, 'dandelion': 2, 'sunflowers': 3, 'daisy': 4}

In [5]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(10),
    transforms.ColorJitter(brightness=0.1, hue=0.1)
])

In [6]:
config = {
    "batch_size" : 32,
    "image_size" : (224,224),
    "channels" : 3,
    "shuffle" : True,
    "channel_first" : True,
    "transforms" : preprocess
}

train_datastream = dataloader.data_generator(
    mode="train",
    **config
)

test_datastream = dataloader.data_generator(
    mode="test",
    **config
)

In [7]:
X, y = next(train_datastream)
X.shape, y.shape

2022-05-19 19:50:53.608269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-19 19:50:53.608307: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-19 19:50:53.608335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ML-RefVm-812132): /proc/driver/nvidia/version does not exist
2022-05-19 19:50:53.608671: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [8]:
X.max(), X.min(), X.mean(), X.std()

(tensor(1.), tensor(0.), tensor(0.3841), tensor(0.2797))

In [9]:
import torchvision
resnet = torchvision.models.resnet18(pretrained=True)

In [10]:
yhat = resnet(X)

In [11]:
yhat.shape

torch.Size([32, 1000])

In [12]:
for name, param in resnet.named_parameters():
    param.requires_grad = False

In [13]:
import torch.nn as nn
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(256, 5)
)

In [14]:
yhat = resnet(X)

In [15]:
(len(dataloader) * 0.8) // 32

91.0

In [16]:
from src.models import train_model
resnet = train_model.trainloop(
    epochs = 25,
    model = resnet,
    optimizer= torch.optim.Adam,
    learning_rate=0.01,
    loss_fn=nn.CrossEntropyLoss(),
    train_dataloader=train_datastream,
    test_dataloader=test_datastream,
    log_dir="../../models/resnet",
    eval_steps=15,
    train_steps=90,
    patience=2,
    factor=0.5,
)

2022-05-19 19:51:38.034 | INFO     | src.data.data_tools:dir_add_timestamp:208 - Logging to ../../models/resnet/20220519-1951
100%|██████████| 90/90 [01:35<00:00,  1.06s/it]
2022-05-19 19:53:33.088 | INFO     | src.models.train_model:trainloop:99 - Epoch 0 train 1.1076 test 0.5476 acc 0.7896
100%|██████████| 90/90 [01:32<00:00,  1.02s/it]
2022-05-19 19:55:18.510 | INFO     | src.models.train_model:trainloop:99 - Epoch 1 train 0.5748 test 0.4959 acc 0.8146
100%|██████████| 90/90 [01:32<00:00,  1.02s/it]
2022-05-19 19:57:03.710 | INFO     | src.models.train_model:trainloop:99 - Epoch 2 train 0.5393 test 0.5021 acc 0.8000
100%|██████████| 90/90 [01:32<00:00,  1.03s/it]
2022-05-19 19:58:49.583 | INFO     | src.models.train_model:trainloop:99 - Epoch 3 train 0.5546 test 0.9287 acc 0.6854
100%|██████████| 90/90 [01:31<00:00,  1.02s/it]
2022-05-19 20:00:34.800 | INFO     | src.models.train_model:trainloop:99 - Epoch 4 train 0.5504 test 0.4293 acc 0.8396
100%|██████████| 90/90 [01:40<00:00,  1